In [ ]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main():
    env = gym.make('CartPole-v1')
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False

        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)      
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask))
            s = s_prime

            score += r
            if done:
                break
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()

n_episode :20, score : 24.5, n_buffer : 490, eps : 7.9%
n_episode :40, score : 20.9, n_buffer : 909, eps : 7.8%
n_episode :60, score : 23.0, n_buffer : 1369, eps : 7.7%
n_episode :80, score : 22.9, n_buffer : 1828, eps : 7.6%
n_episode :100, score : 16.8, n_buffer : 2164, eps : 7.5%
n_episode :120, score : 13.5, n_buffer : 2434, eps : 7.4%
n_episode :140, score : 10.4, n_buffer : 2642, eps : 7.3%
n_episode :160, score : 9.7, n_buffer : 2836, eps : 7.2%
n_episode :180, score : 10.2, n_buffer : 3039, eps : 7.1%
n_episode :200, score : 11.9, n_buffer : 3277, eps : 7.0%
n_episode :220, score : 20.7, n_buffer : 3691, eps : 6.9%
n_episode :240, score : 15.5, n_buffer : 4001, eps : 6.8%
n_episode :260, score : 22.3, n_buffer : 4447, eps : 6.7%
n_episode :280, score : 44.6, n_buffer : 5339, eps : 6.6%
n_episode :300, score : 130.5, n_buffer : 7949, eps : 6.5%
n_episode :320, score : 205.9, n_buffer : 12067, eps : 6.4%
n_episode :340, score : 261.9, n_buffer : 17305, eps : 6.3%
n_episode :360, 

n_episode :2740, score : 409.6, n_buffer : 50000, eps : 1.0%
n_episode :2760, score : 209.7, n_buffer : 50000, eps : 1.0%
n_episode :2780, score : 316.9, n_buffer : 50000, eps : 1.0%
n_episode :2800, score : 424.6, n_buffer : 50000, eps : 1.0%
n_episode :2820, score : 445.4, n_buffer : 50000, eps : 1.0%
n_episode :2840, score : 392.6, n_buffer : 50000, eps : 1.0%
n_episode :2860, score : 468.6, n_buffer : 50000, eps : 1.0%
n_episode :2880, score : 266.9, n_buffer : 50000, eps : 1.0%
n_episode :2900, score : 96.3, n_buffer : 50000, eps : 1.0%
n_episode :2920, score : 121.4, n_buffer : 50000, eps : 1.0%
n_episode :2940, score : 210.3, n_buffer : 50000, eps : 1.0%
n_episode :2960, score : 307.4, n_buffer : 50000, eps : 1.0%
n_episode :2980, score : 370.4, n_buffer : 50000, eps : 1.0%
n_episode :3000, score : 449.6, n_buffer : 50000, eps : 1.0%
n_episode :3020, score : 350.0, n_buffer : 50000, eps : 1.0%
n_episode :3040, score : 416.1, n_buffer : 50000, eps : 1.0%
n_episode :3060, score : 

n_episode :5440, score : 461.9, n_buffer : 50000, eps : 1.0%
n_episode :5460, score : 447.8, n_buffer : 50000, eps : 1.0%
n_episode :5480, score : 470.1, n_buffer : 50000, eps : 1.0%
n_episode :5500, score : 489.3, n_buffer : 50000, eps : 1.0%
n_episode :5520, score : 452.4, n_buffer : 50000, eps : 1.0%
n_episode :5540, score : 462.0, n_buffer : 50000, eps : 1.0%
n_episode :5560, score : 436.6, n_buffer : 50000, eps : 1.0%
n_episode :5580, score : 487.2, n_buffer : 50000, eps : 1.0%
n_episode :5600, score : 500.0, n_buffer : 50000, eps : 1.0%
n_episode :5620, score : 468.7, n_buffer : 50000, eps : 1.0%
n_episode :5640, score : 387.9, n_buffer : 50000, eps : 1.0%
n_episode :5660, score : 420.2, n_buffer : 50000, eps : 1.0%
n_episode :5680, score : 434.4, n_buffer : 50000, eps : 1.0%
n_episode :5700, score : 489.8, n_buffer : 50000, eps : 1.0%
n_episode :5720, score : 428.8, n_buffer : 50000, eps : 1.0%
n_episode :5740, score : 460.9, n_buffer : 50000, eps : 1.0%
n_episode :5760, score :

n_episode :8140, score : 471.4, n_buffer : 50000, eps : 1.0%
n_episode :8160, score : 483.9, n_buffer : 50000, eps : 1.0%
n_episode :8180, score : 459.9, n_buffer : 50000, eps : 1.0%
n_episode :8200, score : 500.0, n_buffer : 50000, eps : 1.0%
n_episode :8220, score : 459.2, n_buffer : 50000, eps : 1.0%
n_episode :8240, score : 483.6, n_buffer : 50000, eps : 1.0%
n_episode :8260, score : 441.7, n_buffer : 50000, eps : 1.0%
n_episode :8280, score : 354.4, n_buffer : 50000, eps : 1.0%
